In [ ]:
# # unzip dataset
# !pwd
# !ls -lh ../0-data/0-dataset
# !ls -lh ../0-data/1-prepare_data
# # mouse 10090
# !gunzip -c ../0-data/0-dataset/10090.protein.links.v11.0.txt.gz > ../0-data/1-prepare_data/10090-ppi.txt
# !gunzip -c ../0-data/0-dataset/uniprot-filtered-organism__Mus+musculus+\(Mouse\)+\[10090\]_+AND+revie--.xlsx.gz > ../0-data/1-prepare_data/10090-uniprot.xlsx
# # human 9606
# !gunzip -c ../0-data/0-dataset/9606.protein.links.v11.0.txt.gz > ../0-data/1-prepare_data/9606-ppi.txt
# !gunzip -c ../0-data/0-dataset/uniprot-filtered-organism__Homo+sapiens+\(Human\)+\[9606\]_+AND+review--.xlsx.gz > ../0-data/1-prepare_data/9606-uniprot.xlsx

# !ls -lh ../0-data/1-prepare_data

# !mkdir ./temp  
# %cd ./temp
# !gunzip -c ../../0-data/0-dataset/uniprot-filtered-organism__Homo+sapiens+\(Human\)+\[9606\]_+AND+review--.fasta.gz > ./9606.fa
# !gunzip -c ../../0-data/0-dataset/uniprot-filtered-organism__Mus+musculus+\(Mouse\)+\[10090\]_+AND+revie--.fasta.gz > ./10090.fa
# !diamond makedb --in ./9606.fa  -d 9606 && rm ./9606.fa
# !diamond makedb --in ./10090.fa -d 10090 && rm ./10090.fa
# !gunzip -c ../../0-data/0-dataset/uniprot-filtered-organism__Homo+sapiens+\(Human\)+\[9606\]_+AND+review--query.fasta.gz > ./9606-query.fa
# !gunzip -c ../../0-data/0-dataset/uniprot-filtered-organism__Mus+musculus+\(Mouse\)+\[10090\]_+AND+revie--qurey.fasta.gz > ./10090-query.fa

# # multiple sequence alignment
# !diamond blastp -q ./9606-query.fa -d 9606.dmnd  -o ../../0-data/1-prepare_data/9606-diamod.tsv --very-sensitive -b8 -c1
# !diamond blastp -q ./10090-query.fa -d 10090.dmnd  -o ../../0-data/1-prepare_data/10090-diamod.tsv --very-sensitive -b8 -c1

# !ls -lh ../../0-data/1-prepare_data/
# %cd ../
# !rm -r ./temp/

In [ ]:
""" 
命名规范
    模块与包：module_name, package_name
    类名与异常：ClassName, ExceptionName
    方法名：method_name, function_name
    变量：instance_var_name, function_parameter_name, local_var_name, GLOBAL_VAR_NAME

作用域：
    protected: _protected_var, _protected_function, _ProtectedClass
    private: __
"""

In [3]:
import pandas as pd
import numpy as np
import os
import pickle
import collections

from tqdm import tqdm
from pandas.core.frame import DataFrame, Series
from typing import Dict, Tuple, Sequence

# output control
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option("max_columns", 1000)
pd.set_option("max_row", 300)
pd.set_option("display.float_format", lambda x: '%.5f' % x)

In [12]:
BASE_DIR = "../0-data/1-prepare_data/"
OUT_DIR = "../0-data/2-build_graphs/"
OBO_NAME = "../0-data/0-dataset/go.obo"
RESERVE_PPI = False
ALPHA_FACTOR = 1500

INPUT_DATASET = {
    "human": {
        "id": "9606",
        "ppi": "9606-ppi.txt",
        "uniprot": "9606-uniprot.xlsx",
        "diamond": "9606-diamond.tsv"
    },
    "mouse": {
        "id": "10090",
        "ppi": "10090-ppi.txt",
        "uniprot": "10090-uniprot.xlsx",
        "diamond": "10090-diamond.tsv"
    }
}

In [5]:
# handle single type of input
def handle_uniprot(df_uniprot: DataFrame, reserve_ppi: bool=False) -> DataFrame:
    df_uniprot.columns = ['uniprot_id', 'string_id', 'sequence', 'labels']
    # 清洗 string_id 字段
    df_uniprot['string_id'] = df_uniprot['string_id'].map(lambda x: x.strip(';') if isinstance(x, str) else x)
    
    # 由于部分对比方法无法处理无PPI信息的数据，因此去除空字段
    # 注意本方法支持无PPI信息的数据，注释掉此句可以保留这些节点
    if not reserve_ppi:
        df_uniprot = df_uniprot.drop(df_uniprot[df_uniprot['string_id'].map(lambda x: False if isinstance(x, str) else True)].index).reset_index(drop=True)
    
    return df_uniprot


def handle_ppi(df_ppi: DataFrame, string2uniprot: dict, reserve_ppi: bool=False) -> DataFrame:
    # 映射函数
    get_uniprot = lambda x: string2uniprot.get(x, float('nan'))
    # 保留所有PPI数据时使用此函数，对于存在映射的，优先使用uniprotid，否则使用stringid
    get_uniprot_reserve_x = lambda x: string2uniprot.get(x, x)
    get_fun = get_uniprot_reserve_x if reserve_ppi else get_uniprot
    
    # 清洗 ppi 数据
    df_ppi['protein1'] = df_ppi['protein1'].map(lambda x: get_fun(x))
    df_ppi['protein2'] = df_ppi['protein2'].map(lambda x: get_fun(x))
    
    # df_ppi = df_ppi.drop(df_ppi[df_ppi['protein1'].map(lambda x: False if isinstance(x, str) else True)].index).reset_index(drop=True)
    # df_ppi = df_ppi.drop(df_ppi[df_ppi['protein2'].map(lambda x: False if isinstance(x, str) else True)].index).reset_index(drop=True)
    df_ppi = df_ppi.dropna().reset_index(drop=True)
    
    return df_ppi


def handle_diamond(df_diamond: DataFrame, uniprot_ids: set, reserve_ppi: bool=False) -> DataFrame:
    df_diamond = df_diamond[['protein1', 'protein2', 'bit_score']].copy(deep=True)
    # 字段映射
    df_diamond['protein1'] = df_diamond['protein1'].map(lambda x: x.split("|")[1].split("-")[0])
    df_diamond['protein2'] = df_diamond['protein2'].map(lambda x: x.split("|")[1].split("-")[0])
    
    # 字段清洗
    df_diamond['protein1'] = df_diamond['protein1'].map(lambda x: x if x in uniprot_ids else float('nan'))
    df_diamond['protein2'] = df_diamond['protein2'].map(lambda x: x if x in uniprot_ids else float('nan'))
    
    df_diamond = df_diamond.dropna().reset_index(drop=True)
    return df_diamond

In [6]:
def preprocessing_data(species: str="human") -> Tuple[DataFrame, DataFrame, DataFrame]:
    """Read source file into memory 
    """
    uniprot = BASE_DIR + INPUT_DATASET[species]['uniprot']
    ppi = BASE_DIR + INPUT_DATASET[species]['ppi']
    diamond = BASE_DIR + INPUT_DATASET[species]['diamond']

    df_uniprot = pd.read_excel(uniprot)[['Entry', 'Cross-reference (STRING)', 'Sequence', 'Gene ontology IDs']]
    df_ppi = pd.read_csv(ppi, sep=' ')
    df_diamond = pd.read_csv(diamond, sep='\t', header=None, names=['protein1', 'protein2', 'sequence identity percentage', 'length', 'mismatches', 'gap openings', 'start1', 'end1', 'start2', 'end2', 'e-value', 'bit_score'])

    df_uniprot = handle_uniprot(df_uniprot, RESERVE_PPI)

    # 映射字典
    string2uniprot = dict(zip(df_uniprot['string_id'], df_uniprot['uniprot_id']))
    df_ppi = handle_ppi(df_ppi, string2uniprot, RESERVE_PPI)

    # 保留 id
    uniprot_ids  = set(df_uniprot['uniprot_id'])
    df_diamond = handle_diamond(df_diamond, uniprot_ids, RESERVE_PPI)
    return df_uniprot, df_ppi, df_diamond

In [7]:
def build_similar_matrix(df_ppi: DataFrame, uniprot2id: dict, n: int, out_prefix: str) -> str:
    
    cnt = 0
    path = "./" + out_prefix + "-ppi-mat.pkl"
    
    if os.path.exists(path):
        return path
    similar_matrix = [[0.]*n for i in range(n)]

    for _, row in tqdm(df_ppi.iterrows(), total = len(df_ppi)):
        cnt += 1
        if (cnt % 1000000 == 0):
            print(row)
        x = uniprot2id[row['protein1']]
        y = uniprot2id[row['protein2']]
        if (similar_matrix[x][y] != 0):
            print(x, " ", y)
        similar_matrix[x][y] += row['combined_score']

    cnt
    with open(path, "wb+") as f:
        pickle.dump(similar_matrix, f)
        
    return path

def build_diamond(df_uniprot: DataFrame, df_diamond: DataFrame, uniprot2id: dict, path: str) -> list:
    with open(path, "rb") as f:
        similar_matrix = pickle.load(f)

    for protein1 in tqdm(df_uniprot['uniprot_id']):
    #     print(protein1)
        df = df_diamond[df_diamond['protein1']==protein1]
        x = uniprot2id[protein1]

        for score in df[df['protein2'] == protein1]['bit_score']:
            similar_matrix[x][x] = max(similar_matrix[x][x], score)

        for _, row in df.iterrows():
            y = uniprot2id[row['protein2']]
            if x != y:
                similar_matrix[x][y] = row['bit_score'] / similar_matrix[x][x] * ALPHA_FACTOR

        similar_matrix[x][x] = 0

#     with open("./9606-similar-mat.pkl", "wb+") as f:
#         pickle.dump(similar_matrix, f)
    return similar_matrix

def build_ppsn(df_uniprot: DataFrame, df_ppi: DataFrame, df_diamond: DataFrame, species: str="human") -> DataFrame:
    
    out_prefix = INPUT_DATASET[species]['id']
    df_uniprot = df_uniprot.reset_index().rename(columns={'index': 'id'})
    uniprot2id = dict(zip(df_uniprot['uniprot_id'], df_uniprot['id']))
    path = build_similar_matrix(df_ppi, uniprot2id, len(df_uniprot), out_prefix)
    similar_matrix = build_diamond(df_uniprot, df_diamond, uniprot2id, path)
    
    protein1 = []
    protein2 = []
    score = []
    n = len(df_uniprot)

    for i in tqdm(range(n)):
        for j in range(n):
            if similar_matrix[i][j] != 0:
                if i == j:
                    print("error")
                    break;
                protein1.append(i)
                protein2.append(j)
                score.append(similar_matrix[i][j])
                
    # protein-protein similarity networks
    ppsn = DataFrame({
        'protein1': protein1,
        'protein2': protein2,
        'score': score
    })
    
    # output result
    ppsn.to_csv(OUT_DIR + out_prefix + "-ppsn-min.csv", index=False, sep='\t')
    df_uniprot.to_csv(OUT_DIR + out_prefix +"-uniprot-min.csv", index=False, sep='\t')
    return ppsn

In [22]:
def resolve_terms():
    
    go_path = OBO_NAME
    out_path = OUT_DIR + "terms.pkl"
    gos = []
    global namespace
    namespace = collections.defaultdict(str)
    is_a = collections.defaultdict(list)
    part_of = collections.defaultdict(list)

    # 根据规则来提取go term ，并依据其之间的依赖关系构建图谱
    with open(go_path,'r')as f:
        for line in f:
            if '[Typedef]' in line:
                break
            if line[:5]=='id: G':                       # 构建 gos
                line=line.strip().split()
                gos.append(line[1])
            elif line[:4]=='is_a':                      # 构建 is_a 关系
                line=line.strip().split()
                is_a[gos[-1]].append(line[1])
            elif line[:4]=='rela' and 'part' in line:   # 构建 partof 关系
                line=line.strip().split()
                part_of[gos[-1]].append(line[2])
            elif line[:5]=='names':                     # 统计子本体
                line=line.strip().split()
                namespace[gos[-1]]=line[1]

    son_of = {
        "GO:0008150": None,
        "GO:0005575": None,
        "GO:0003674": None
    }

    son_of = {**son_of, **is_a}

    for i in part_of:
        son_of[i].extend(part_of[i])

    cross_ontology = 0
    for k in son_of:
        tag = False
        if son_of[k] is None:
            continue
        for j in son_of[k]:
            if namespace[k]!=namespace[j]:
    #             print("error: child->{}, father->{}".format(k, j))
                tag = True
        if tag:
            cross_ontology += 1
            
    with open(out_path, "wb+") as f:
        pickle.dump([son_of, namespace], f)
    
    print("Cross-ontology connection in prat_of: ", cross_ontology)
    print("Total terms: ", len(gos))         # 47210
    print("is_a relation: ", len(is_a))        # 44082
    print("part_of relation: ", len(part_of))     # 8295
    print(len(namespace))   # 47210
    print("output terms: ", len(son_of))

In [23]:
resolve_terms()

Cross-ontology connection in prat_of:  1165
Total terms:  47210
is_a relation:  44082
part_of relation:  8295
47210
output terms:  44085


In [8]:
# TODO: build ppsn which retains all information of ppi 
for species in INPUT_DATASET.keys():
    print(species)
    df_uniprot, df_ppi, df_diamond = preprocessing_data(species)

    print("*"*35, species, "*"*35, ": ")
    df_uniprot.shape
    df_ppi.shape
    df_diamond.shape

    df_uniprot.head()
    df_ppi.head()
    df_diamond.head()

    ppsn = build_ppsn(df_uniprot, df_ppi, df_diamond, species)

human


/home/zhangxs/.local/anaconda3/envs/dgl/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  


*********************************** human *********************************** : 


(18560, 4)

(11098152, 3)

...

(191350, 3)

,uniprot_id,string_id,sequence,labels
0,Q8VCZ3,10090.ENSMUSP00000097701,MPRHCSAAGCCTRDTRETRNRGISFHRLPKKDNPRRGLWLANCQRL...,GO:0000122; GO:0001226; GO:0003677; GO:0005634...
1,Q62264,10090.ENSMUSP00000042988,MQVLTKRYPKNCLLTVMDRYSAVVRNMEQVVMIPSLLRDVQLSGPG...,GO:0005654; GO:0005829; GO:0006629; GO:0009617...
2,Q6QNU9,10090.ENSMUSP00000074381,MGRYWLLPGLLLSLPLVTGWSTSNCLVTEGSRLPLVSRYFTFCRHS...,GO:0002224; GO:0004888; GO:0005887; GO:0006954...
3,Q8BHE4,10090.ENSMUSP00000140027,MKRSLQALYCQLLSFLLTLALTKALVLAVHEPSPRESLQTLPSGSP...,GO:0005769; GO:0006898; GO:0008090; GO:0010008...
4,Q9JLF7,10090.ENSMUSP00000106625,MACQLDLLIGVIFMASPVLVISPCSSDGRIAFFRGCNLTQIPWILN...,GO:0002224; GO:0002755; GO:0004888; GO:0005149...


,protein1,protein2,combined_score
0,Q9DC51,Q9WUN2,183
1,Q9DC51,P11031,155
2,Q9DC51,Q9JKV2,258
3,Q9DC51,P14685,165
4,Q9DC51,Q8BHC1,192


,protein1,protein2,bit_score
0,Q8VCZ3,Q8VCZ3,622.00000
1,Q8VCZ3,Q9D305,62.00000
2,Q62264,Q62264,294.00000
3,Q62264,Q9CQ20,71.60000
4,Q6QNU9,Q6QNU9,1714.00000


100%|██████████| 16420/16420 [00:32<00:00, 503.97it/s]


In [167]:
INPUT_DATASET.keys()

dict_keys(['human', 'mouse'])